In [7]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/luxe/brands/mac/c/3899?"
params = "ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "M.A.C",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/luxe/brands/mac/c/3899?ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C
Scraping Page 2: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=2&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop
Scraping Page 3: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=3&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop
Scraping Page 4: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=4&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop
Scraping Page 5: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=5&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop
Scraping Page 6: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=6&sort=popularity&ptype=brand&id=3899&root=brand_menu,top_brands,M.A.C&eq=desktop
Scraping Page 7: https://www.nykaa.com/luxe/brands/mac/c/3899?page_no=7&sort=popularity&ptype=brand&id=3899&root=brand_menu,t

In [11]:
# print(f"Products: \n {products}")

In [10]:
mac = pd.DataFrame(products)
mac

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,M.A.C,M.A.C MACximal Satin Lipstick - Creme In Your ...,4.1/5,44ratings&16reviews,₹2550,N/A,No Discount,None,In Stock
1,M.A.C,M.A.C Strobe Cream Mini - Pinklite Illuminatin...,4.4/5,15575ratings&1342reviews,₹1400,N/A,No Discount,BESTSELLER,In Stock
2,M.A.C,M.A.C Studio Fix Powder Plus Compact Foundatio...,4.4/5,18824ratings&1835reviews,₹3900,N/A,No Discount,BESTSELLER,In Stock
3,M.A.C,M.A.C MACximal Mini Matte Nude Lipstick - Mehr...,4.4/5,30727ratings&5295reviews,₹1650,N/A,No Discount,BESTSELLER,In Stock
4,M.A.C,M.A.C Prep + Prime Fix+ (Alcohol Free) Setting...,4.5/5,285ratings&19reviews,₹2700,N/A,No Discount,BESTSELLER,In Stock
...,...,...,...,...,...,...,...,...,...
244,M.A.C,M.A.C Sparkler Eyeshadow- Holiday Collection Z...,5/5,3ratings,₹2400,N/A,No Discount,None,Out Of Stock
245,M.A.C,M.A.C Lash - 7(1 pair),4.4/5,5ratings&1reviews,₹1400,N/A,No Discount,None,Out Of Stock
246,M.A.C,M.A.C Powder Kiss Velvet Blur Slim Stick- Holi...,4.3/5,3ratings&1reviews,₹3400,N/A,No Discount,None,Out Of Stock
247,M.A.C,M.A.C Lash - 36(1 pair),4.8/5,11ratings&1reviews,₹1500,N/A,No Discount,None,Out Of Stock


In [12]:
# mac.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/M.A.C Products_Nykaa.csv", index = False)